# 데이터 집계(aggregation)과 group 연산

- 데이터 집합 분류/그룹별 함수 적용
- pandas는 *groupby* interface 제공
    - 하나 이상의 key를 이용한 pandas 객체의 조각 나누기
    - group summary statistics
    - Apply within-group transformations: normalization, linear regression, rank, or subset selection
    - pivot tables and cross-tabulations
    - quantile analysis and other statistical group analyses
- 참고문헌
    - Python for Data Analysis (Ch.10), Wes Mckinney, O'Reily, 2nd Edi., 2017.
    - https://datascienceschool.net/view-notebook/6e71dbff254542d9b0a054a7c98b34ec/

# 1. GroupBy Mechanics
- *split-apply-combine* for group operation
![그림10-1](img/fig10-1.PNG)

- **grouping key**
    - A list or array of values that is the same length as the axis being grouped
    - A value indicating a column name in a DataFrame
    - A dict or Series giving a correspondence between the values on the axis being grouped and the group names
    - A function to be invoked on the axis index or the individual labels in the index

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
list('ABCD')

['A', 'B', 'C', 'D']

In [ ]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

In [ ]:
# to compute the mean of the data1 column using the labels from key1.
grouped = df['data1'].groupby(df['key1'])
grouped

In [ ]:
grouped.mean()

In [ ]:
dir(grouped)

In [ ]:
# 두 개의 key로 group
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

In [ ]:
# multiindex일 때 inner-most-level의 key를 column key로
means.unstack()

In [ ]:
means.unstack(level=0)

In [ ]:
# 길이가 같으면 series도 groupby 연산 가능
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

In [ ]:
df.groupby('key1').mean()

In [ ]:
df.groupby(['key1', 'key2']).mean()
# 결과: key2에 대한 평균 없음 => key2는 문자열

In [ ]:
# size(): group의 크기
df.groupby(['key1', 'key2']).size()

## 1.1 Iterating Over Groups
- iteration 지원
- (group 이름, 데이터 묶음)의 tuple을 반환

In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

In [ ]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

In [ ]:
list(df.groupby('key1'))

In [ ]:
# groupby 결과를 {group이름: data묶음}의 dictionary화
pieces = dict(list(df.groupby('key1')))
pieces

In [ ]:
pieces['b']

In [ ]:
df.dtypes

In [ ]:
# df.dtypes에 따라 group
grouped = df.groupby(df.dtypes, axis=1)

In [ ]:
for dtype, group in grouped:
    print(dtype)
    print(group)

## 1.2 Selecting a Column or Subset of Columns

![column선택](img/p293_10.1.2.PNG)

In [ ]:
df.groupby('key1')['data1'].mean()

In [ ]:
# 배열이나 list로 column을 넘기면 DataFrame을 return
df.groupby(['key1', 'key2'])[['data2']].mean()

In [ ]:
# 단일 값으로 column 이름 하나만 지정하면 SeriesGroupBy 객체 반환
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [ ]:
s_grouped.mean()

## 1.3 Grouping with Dictionarys and Series

In [ ]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

In [ ]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

In [ ]:
# 관련성 있는 group 목록
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
mapping

In [ ]:
# 관련성 있는 group 목록에서 group별로 column 을 합함.
by_column = people.groupby(mapping, axis=1)
by_column.sum()

In [ ]:
map_series = pd.Series(mapping)
map_series

In [ ]:
people.groupby(map_series, axis=1).count()

## 1.4 Grouping with Functions
- python 함수를 사용하여 group별 mapping
- 색인 값 하나마다 함수 호출
- 함수 반환 값이 group의 이름으로 사용

In [ ]:
# 이름의 길이별로 grouping
people.groupby(len).sum()

In [ ]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

## 1.5 Grouping by Index Levels

In [ ]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
columns

In [ ]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

In [ ]:
hier_df.groupby(level='cty', axis=1).count()

# 2. Data 집계(aggregation)

- 데이터 수집: data transformation that produces scalar values from arrays.
![표10-1](img/table10-1.PNG)

In [ ]:
df

In [ ]:
# series.quantile(0.75): 분위수
# 각 series에 대해 quantile을 구하고 이들을 합쳐 반환
# a : -0.204707 ~ 1.965781 사이에서 75%에 해당되는 수
# b : -0.555730 ~ -0.519439 사이에서 75%에 해당되는 수
grouped = df.groupby('key1')
grouped['data1'].quantile(0.75)

In [ ]:
# 함수를 이용한 data 수집: agg, aggregate method 
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

In [ ]:
grouped.describe()

## 2.1 Column-Wise and Multiple Function Application

In [ ]:
tips = pd.read_csv('../examples/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

In [ ]:
grouped = tips.groupby(['day', 'smoker'])

In [ ]:
# 표 10-1의 함수 이름을 agg method에 전달
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

In [ ]:
# column 이름: 적용 함수 이름
grouped_pct.agg(['mean', 'std', peak_to_peak])

In [ ]:
# column 이름 지정: (이름, 함수)
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

In [ ]:
# 여러 개의 함수를 column에 적용
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

In [ ]:
result['tip_pct']

In [ ]:
# 각 column 이름 지정
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

In [ ]:
# 각 column 마다 다른 함수 적용
grouped.agg({'tip' : np.max, 'size' : 'sum'})

In [ ]:
# 각 column 마다 다른 함수 적용
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

## 2.2 Returning Aggregated Data Without Row Indexes
- as_index=False 옵션

In [ ]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

# 3. Apply: General split-apply-combine
![그림10-1](img/fig10-1.png)

In [ ]:
# top 함수: tip_pct column 을 정렬하고 마지막 큰 값 5개 반환
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

In [ ]:
# smoker group별 tip_pnt가 높은 상위 5개
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

In [ ]:
result.unstack('smoker')

f = lambda x: x.describe()
grouped.apply(f)

## 3.1 Group Keys 생략
- group_keys=False 옵션

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

## 3.2 Quantile and Bucket Analysis
- cut: 데이터구간, qcut: 분위수 구간 => category 객체 생성
- 구간 data를 groupby 와 결합

In [ ]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)    # 4개 구간으로 나눔
quartiles[:10]

In [ ]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data1.groupby(quartiles)
grouped.apply(get_stats).unstack()

In [ ]:
# data1으로 나누어진 구간에서 data2 group 통계
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

In [ ]:
# Return quantile numbers (데이터를 분위수 10개로 나눔)
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

## 3.3 예제1: Filling Missing Values with Group-Specific Values
- fillna

In [ ]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

#### Extended Slices
- s[::2] : 'nothing for the first argument, nothing for the second, and jump by third step'. 
- L = range(10)
- L[::2] => [0, 2, 4, 6, 8]
- L[::-1] => [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [ ]:
s.fillna(s.mean())

In [ ]:
# group 마다 다른 값으로 fill: apply
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

In [ ]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

In [ ]:
data.groupby(group_key).mean()

In [ ]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

In [ ]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

## 3.4 예제2: Random Sampling and Permutation
- sample

In [ ]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [ ]:
deck[:13]

In [ ]:
# deck에서 임의의 갯수 n개 추출
def draw(deck, n=5):
    return deck.sample(n)
draw(deck,10)

In [ ]:
# 카드 suit 으로 group
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

## 3.5 예제3: Group Weighted Average and Correlation

In [ ]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

In [ ]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

In [ ]:
close_px = pd.read_csv('../examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [ ]:
rets = close_px.pct_change().dropna()

In [ ]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

In [ ]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

## 3.6 예제4: Group-Wise Linear Regression

In [ ]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

# 4. Pivot Tables and Cross-Tabulation

- pivot table: data summariztion tool
- groupby 와 reshape 결합
- pivot_table 함수

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

![표10-2](img/table10-2.png)

## Cross-Tabulations: Crosstab

In [ ]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [ ]:
data

In [ ]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS